In [ ]:
import pandas as pd
import numpy as np

### Preprocessing for all possible TABLES

In [ ]:
df = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")

In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport
profile = ProfileReport(df)

In [ ]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [ ]:
def fun(x, mode): 
    altitud = str(x).split('-')
    if altitud[0] == "nan":
        if mode == "nan_1":
            return -1
        return np.nan
    if mode == "nan_1":
            return x
    if mode == "min":
        return int(altitud[0])
    if mode == "diff":
        if len(altitud) == 1:
            return 0
        else:
            return int(altitud[1])-int(altitud[0])

df["ALTITUD_MIN"] = df["ALTITUD"].apply(fun, args=("min",))
df["ALTITUD_DIF"] = df["ALTITUD"].apply(fun, args=("diff",))
df["ALTITUD"] = df["ALTITUD"].apply(fun, args=("nan_1",))

In [ ]:
df_estacion_gb_mean = df.groupby(by="ID_ESTACION").mean()
df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)

In [ ]:
df_estacion_gb_mean.reset_index(inplace=True)
df_estacion_gb_mode.reset_index(inplace=True)

In [ ]:
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "0", "ALTITUD"] = 390
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "4", "ALTITUD"] = 520

In [ ]:
for i in df["ID_ESTACION"].unique():
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_MIN"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_MIN"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_DIF"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_DIF"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD"] = df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == i,"ALTITUD"].values[0]

In [ ]:
df.loc[df["MODO"]=="1", "MODO"]  = "0"
df.loc[df["MODO"]=="2", "MODO"]  = "1"

Al fer servir producció com a feature, no podrem calcular la superfície de CAMPAÑA. S'haurà de fer manualment.

In [ ]:
df = df.drop(index=df[df["CAMPAÑA"] == "22"].index)

### 1 - Table without SUPERFICIE

In [ ]:
df_temporal = df.drop(axis = 1, columns = ["SUPERFICIE"])

In [ ]:
import category_encoders as ce

encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])
df_encoded = encoder.fit_transform(df_temporal)
encoder = ce.OrdinalEncoder(cols = ["CAMPAÑA"])
df_encoded = encoder.fit_transform(df_temporal)

In [ ]:
df_temporal.to_csv("D_T_1.csv")
df_encoded.to_csv("D_T_1_encoded.csv")

### 2 - Table without CAMPAÑA and SUPERFICIE

In [ ]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA", "SUPERFICIE"])

In [ ]:
import category_encoders as ce
encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])
df_encoded = encoder.fit_transform(df_temporal)

In [ ]:
df_temporal.to_csv("D_T_2.csv")
df_encoded.to_csv("D_T_encoded_2.csv")

### 3 - Table without CAMPAÑA. SUPERFICIE PREDICTION

#### X, y, X_train, y_train, X_test, y_test

In [ ]:
import category_encoders as ce
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD", "COLOR", "TIPO", "MODO"])

In [ ]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA"])

df_encoded = encoder.fit_transform(df_temporal)

train = df_encoded[(df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0)]
to_predict = df_encoded[~((df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0))]

X = train.drop(axis = 1, columns = ["SUPERFICIE"])
y = train["SUPERFICIE"]
to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=3)

In [ ]:
"""from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

normalizer = StandardScaler()
norm_X = normalizer.fit_transform(X)
norm_to_predict = normalizer.transform(to_predict)"""

#### xgboost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
gsc = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [300, 500, 1000],
                        "learning_rate": [0.15],
                        "max_depth": [40, 60, 100],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='r2', verbose=0, n_jobs=-1)

grid_result = gsc.fit(X, y)

In [ ]:
print(grid_result.best_params_)
df_grid = pd.DataFrame(grid_result.cv_results_)
df_grid[df_grid["mean_test_score"]>0.86]

#### Evaluating r2score

In [ ]:
import xgboost as xgb
params_model = {'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.15, 'max_depth': 60, 'min_child_weight': 1, 'n_estimators': 1000}
model = xgb.XGBRegressor(**params_model)
model.fit(X_train,y_train)
model.score(X_test, y_test)

#### Predicting rows with no SUPERFICIE

In [29]:
#params_model = grid_result.best_params_
params_model = {'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.15, 'max_depth': 60, 'min_child_weight': 1, 'n_estimators': 1000}
model = xgb.XGBRegressor(**params_model)
model.fit(X,y)
predicted = model.predict(to_predict)

In [30]:
final_predict = df[~((df["PRODUCCION"] != 0) & (df["SUPERFICIE"] != 0))]
final_predict["SUPERFICIE"] = predicted
final_train = df[(df["PRODUCCION"] != 0) & (df["SUPERFICIE"] != 0)]

C:\Users\xavid\AppData\Local\Temp\ipykernel_18596\193630758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_predict["SUPERFICIE"] = predicted


In [35]:
final_df = pd.concat([final_predict, final_train])

In [36]:
final_df.to_csv("./ds_tractats/df_train_tractat.csv", index = False)

OSError: Cannot save file into a non-existent directory: 'ds_tractats'